Create evaluations of the likelihood for plotting purposes.
Evaluate likelihood on a grid of prm, ic combinations.

In [4]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [5]:
using DifferentialEquations, CSV, DataFrames, Plots

In [6]:
function klausmeier!(du,u,p,t)
 du[1] = -u[1] - u[1] * u[2]^2 + p[1]
 du[2] = u[1] * u[2]^2 - p[2] * u[2]
end

klausmeier! (generic function with 1 method)

In [7]:
abstract type AbstractHyperprm end

struct Hyperprm <: AbstractHyperprm
    w0::Float64
    n0::Float64
    a::Float64
    m::Float64
    M::Float64
end

In [8]:
abstract type AbstractGridprm end

struct Gridprm <: AbstractGridprm
    x_end::Float64
    y_end::Float64
    len::Int64
end

In [9]:
function sol_klausmeier(hprm::Hyperprm)
    u0 = [hprm.w0; hprm.n0]
    p = [hprm.a; hprm.m]
    tspan = (0.0, hprm.M-1)

    prob = ODEProblem(klausmeier!, u0, tspan, p)
    sol = solve(prob,
        saveat=1.0  # consider specific time points
    )

    return DataFrame(time=sol.t, w=sol[1, :], n=sol[2, :])
end

sol_klausmeier (generic function with 1 method)

In [107]:
function create_grid(gprm::Gridprm)
    x_vals = range(0, gprm.x_end; length=gprm.len)
    y_vals = range(0, gprm.y_end; length=gprm.len)
    
    grid = [(x, y) for x in x_vals, y in y_vals] 
    return grid
end

create_grid (generic function with 2 methods)

In [67]:
function compute_ll(true_val::DataFrame, pred_val::DataFrame)
    return 1
end

compute_ll (generic function with 1 method)

Consider x in (0,2), y in (0,4.2).
As in the reference paper we evaluate len=11

In [ ]:
function gen_ll_evals_for_hprm_comb(hprm_true::Hyperprm, noise::Float64, gprm::Gridprm)

    grid = create_grid(gprm)
    sol_true = sol_klausmeier(hprm_true) # returns df
    #TODO: incorporate random noise to true data

    ll = zeros(size(grid)[1], size(grid)[1])

    for i in range(1, size(grid)[1])
        for j in range(1, size(grid)[1])
            print([i,j])

            #eval model for each point on grid
            pt = grid[i,j]
            hprm = Hyperprm(hprm_true.w0, pt[2], pt[1], hprm_true.m, hprm_true.M) #w0,n0,a,m,M
            sol_pred = sol_klausmeier(hprm)

            #eval likelihood
            ll_val = compute_ll(sol_true, sol_pred)
            
            ll[i,j] = ll_val
        end
    end

    #return data frame
    a_vals = string.(collect(range(0, gprm.x_end; length=gprm.len)))
    df_ll = DataFrame(ll, a_vals)
    
    return df_ll
end


#TODO: store sols each in csv with correct format and name

gen_ll_evals_for_hprm_comb (generic function with 2 methods)

In [109]:
gen_ll_evals_for_hprm_comb(hprm, 0.0)

[1, 1][1, 2][1, 3][1, 4][1, 5][1, 6][1, 7][1, 8][1, 9][1, 10][1, 11][2, 1][2, 2][2, 3][2, 4][2, 5][2, 6][2, 7][2, 8][2, 9][2, 10][2, 11][3, 1][3, 2][3, 3][3, 4][3, 5][3, 6][3, 7][3, 8][3, 9][3, 10][3, 11][4, 1][4, 2][4, 3][4, 4][4, 5][4, 6][4, 7][4, 8][4, 9][4, 10][4, 11][5, 1][5, 2][5, 3][5, 4][5, 5][5, 6][5, 7][5, 8][5, 9][5, 10][5, 11][6, 1][6, 2][6, 3][6, 4][6, 5][6, 6][6, 7][6, 8][6, 9][6, 10][6, 11][7, 1][7, 2][7, 3][7, 4][7, 5][7, 6][7, 7][7, 8][7, 9][7, 10][7, 11][8, 1][8, 2][8, 3][8, 4][8, 5][8, 6][8, 7][8, 8][8, 9][8, 10][8, 11][9, 1][9, 2][9, 3][9, 4][9, 5][9, 6][9, 7][9, 8][9, 9][9, 10][9, 11][10, 1][10, 2][10, 3][10, 4][10, 5][10, 6][10, 7][10, 8][10, 9][10, 10][10, 11][11, 1][11, 2][11, 3][11, 4][11, 5][11, 6][11, 7][11, 8][11, 9][11, 10][11, 11]

Row,0.0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
hprm = Hyperprm(0,1,1,0.45,1000)
gprm = Gridprm(2,4.2,11)

Gridprm(2.0, 4.2, 11)

?????? was nehmen als w0